In [1]:
from tensorflow.keras.callbacks import Callback
import pandas as pd
import random 
import numpy as np
import math
from matplotlib import pyplot as plt
from matplotlib.pylab import mpl
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LeakyReLU,Conv1D,Dense
from tcn import TCN,tcn_full_summary
from sklearn.metrics import mean_squared_error # 均方误差
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Input,Model,Sequential

# 音频

In [2]:
def rmse(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [3]:
audio_tcn = tf.keras.models.load_model('audio_1.h5',custom_objects={'TCN':TCN,'rmse':rmse})

In [4]:
audio_path = 'H:/test_data/audio_data/test_audio.csv'
audio_df = pd.read_csv(audio_path,index_col=0)

In [5]:
audio_X_Test=audio_df.iloc[:,:-1]
audio_Y_test=audio_df.iloc[:,-1:]

In [6]:
audio_df = audio_df.iloc[:,:-1]
audio_scaler = MinMaxScaler(feature_range=(-1,1)).fit(audio_df)
audio_X_test=pd.DataFrame(audio_scaler.transform(audio_X_Test))

In [7]:
audio_x_test=audio_X_test.values.reshape([audio_X_test.shape[0],audio_X_test.shape[1],1])  
audio_y_test=audio_Y_test.values

In [8]:
audio_predict = audio_tcn.predict(audio_x_test)
audio_real_y = audio_y_test

# 面部

In [10]:
vidio_tcn = tf.keras.models.load_model('vidio.h5',custom_objects={'TCN':TCN})

In [11]:
vidio_path = 'H:/test_data/vidio_data/test_vidio.csv'
vidio_df = pd.read_csv(vidio_path,index_col=0)

In [12]:
vidio_X_Test=vidio_df.iloc[:,:-1]
vidio_Y_test=vidio_df.iloc[:,-1:]

In [13]:
vidio_df = vidio_df.iloc[:,:-1]
vidio_scaler = MinMaxScaler(feature_range=(-1,1)).fit(vidio_df)
vidio_X_test=pd.DataFrame(vidio_scaler.transform(vidio_X_Test))

In [14]:
vidio_x_test=vidio_X_test.values.reshape([vidio_X_test.shape[0],1,vidio_X_test.shape[1]])  
vidio_y_test=vidio_Y_test.values

In [15]:
vidio_predict = vidio_tcn.predict(vidio_x_test)
vidio_real_y = vidio_y_test

# 决策融合

In [20]:
audio_predict，vidio_predict，vidio_y_test

(47, 1)

In [23]:
df = pd.DataFrame(columns=('audio_Score','vidio_Score','PHQ8_Score'))

In [25]:
for i in range(vidio_y_test.shape[0]):
    df = df.append([{'audio_Score':audio_predict[i][0],'vidio_Score':vidio_predict[i][0],'PHQ8_Score':vidio_y_test[i][0]}],ignore_index=True)

In [27]:
df.head()

,audio_Score,vidio_Score,PHQ8_Score
0,3.030377,7.857666,2.0
1,5.393793,8.807657,3.0
2,3.334128,13.657556,0.0
3,5.721376,10.798936,22.0
4,5.728904,9.253892,15.0


In [ ]:
n_hidden_1 = 64 #隐藏层1的神经元个数
n_hidden_2 = 64 #隐藏层2的神经元个数
n_input = 13 #输入层的个数
n_classes = 1 #输出层的个数（我们只预测房价，就1个值所以输出是1）
training_epochs = 200 #训练次数，总体数据需要循环多少次
batch_size = 10  #每批次要取的数据的量，这里是提取10条数据

In [ ]:
model = Sequential()#先建立一个顺序模型
#向顺序模型里加入第一个隐藏层，第一层一定要有一个输入数据的大小，需要有input_shape参数
#model.add(Dense(n_hidden_1, activation='relu', input_shape=(n_input,)))
model.add(Dense(n_hidden_1, activation='relu', input_dim=n_input)) #这个input_dim和input_shape一样，就是少了括号和逗号
model.add(Dense(n_hidden_2, activation='relu'))
model.add(Dense(n_classes)) #因为我们是预测房价，不是分类，所以最后一层可以不用激活函数

In [ ]:
model.compile(loss='mse', optimizer='rmsprop', metrics=['mae',r2])

In [ ]:
history = model.fit(train_x, train_y, batch_size=batch_size, epochs=training_epochs)

pred_test_y = model.predict(test_x)
print(pred_test_y)